# Querying our GraphQL API

The PostgreSQL database exposes a GraphQL API that one could query directly. Usually it's more convenient to use the caching interface that we use in the flatsurvey scripts themselves to speed up surveys.

In [1]:
from flatsurvey.cache import GraphQL
cache = GraphQL()

## Surfaces with non-dense Orbit Closure

In [2]:
from flatsurvey.jobs import OrbitClosure

We pull all OrbitClosures from the cache where we could not determine that the orbit closure was dense. Since we can currently never determine that the orbit closure is *not* dense, these are the ones where the search was inconclusive, that is, it reported `None` which turns to `null` in this API.

In [3]:
nondense = cache.query(job=OrbitClosure, result_filter="dense: { isNull: true }")

The underlying objects from the database are in `.nodes()`. Each of these nodes corresponds to one `OrbitClosure` computation. The corresponding surface is a pickle that can be recovered with such a call:

In [4]:
nondense.nodes()[0]['surface']()

/tmp/ruth/miniconda/envs/survey/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:1: DeprecationWarning: 
Importing call_method from here is deprecated. If you need to use it, please import it directly from sage.misc.call
See https://trac.sagemath.org/29869 for details.
  from ipykernel.kernelapp import IPKernelApp


Ngon([2, 2, 3])

Let's recover all the surfaces and remove duplicates:

In [5]:
surfaces = set(node['surface']() for node in nondense.nodes())

We remove all the surfaces that we already know about as encoded by `.reference()`.

In [6]:
surfaces = set(surface for surface in surfaces if surface.reference() is None)

Note that this set might appear to contain duplicates since e.g. two quadrilaterals with differently chosen random lengths are printed in the same way. Let's remove these "duplicates" as well.

In [7]:
surfaces = set(str(surface) for surface in surfaces)

Some of the orbit closure computations might not have found the full dimension of the orbit closure because they did not search deep enough. Let's recover all the computations for our surfaces and remove the surfaces where some run reported a dense orbit closure.

We could use `cache.query` again for this. To determine the cached results for a fixed surface, `cache.results()` provides a wrapper around `cache.query` for this.

In [8]:
orbit_closures = {
    name: cache.results(job=OrbitClosure, surface=name) for name in surfaces
}

We filter out all the surfaces where some run determined that the orbit closure was dense.

This could be done by comparing `node['dense']` for each node in `.nodes()`. Here we use `.reduce()` which calls into `OrbitClosure` to combine the results of several runs.

In [9]:
orbit_closure = {
    name: cached for (name, cached) in orbit_closures.items() if cached.reduce() is None
}

In [10]:
orbit_closures

{'Ngon([1, 4, 11])': Cached OrbitClosures,
 'Ngon([3, 4, 13])': Cached OrbitClosures,
 'Ngon([1, 4, 15])': Cached OrbitClosures}

Let's zoom in on one particular surface:

In [11]:
orbit_closures = orbit_closures['Ngon([3, 4, 13])']

Again, `.nodes()` contains the raw data stored in the database.

In [12]:
orbit_closures.nodes()

[{'lot': 6856453675869781000,
  'result': <function GraphQL.resolve.<locals>.restore at 0x7fbcbfa510e0>,
  'command': ['orbit-closure'],
  'dimension': 4,
  'directions': 460,
  'invocation': ['/dev/shm/ruth/survey/flatsurvey/worker/__main__.py',
   'pickle',
   '--base64',
   'eJyVVgd4FFUQTqEuTcQCVuyHSkiAIFiQSDNEThxAzxLXzd7L7ZK93fy7e4GIUSx7SxQVsffeO/besfdesffeu86WI0cIiN9937s3b/q/M/PegjK10VBcJ2e3iNYK/mtUVOFUmBnLdKQkrxJK6lGaUB0lIyps3cw4FbrpioywpazSJOQCgbJUaUlJySiUe+hG6K5F9Gj08NCT0Cum0+jtQSL0EameTIrhiunqKvpG9rNWOmew+0ZbCDkiZGGIrDDdyNsUZkwPzydHx3JGmMLWVTktTEfILVUS+sXBOq6dU92cLSo4J9eyW6WCbEzLObNZV5sMIaH/anUMy2rKNcsZw2pQDAkDUhKH3REG1vEwkLBuXe+6sjwGJbT+CW1AahzLFAFm5rINwpYbdWGkVyaSITEl2MstwnZ0yxyJ9TysT9hAG7QS6rbiMlcxIk2JYjLUlbDhMMLgYvFmy2g1rayuGEVbucOIwR9OmrFaDoYk1sIa82SVKyVEzbKlAqByh2FiGQkbJbTB4fefh409bJJsd7EpYbN6bN5ljtHHXkFhaKhbjS08bEnYSovoSmztYRvCtkyXMT28Gtt5SBCGaUM1Zm1P2CEWrcKOHoYTKkQ7ux5BqAw11EpUeRjZ+dunW00ly3WiGorjSCtTGJVIDWXVGiMjGmxFV0nEH0Geq7uarCrcD1w4GF2cGNCg2NKqOhKqVVluyOmGq5uyzBXqK

To unpickle all the `['result']` objects at once, we can use `.results()`:

In [13]:
orbit_closures.results()

/tmp/ruth/miniconda/envs/survey/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: sage.structure.element.Vector size changed, may indicate binary incompatibility. Expected 40 from C header, got 48 from PyObject
  return f(*args, **kwds)


[GL(2,R)-orbit closure of dimension at least 4 in H_8(12, 2) (ambient dimension 17)]

## Flow Decompositions with Undetermined Components

In [14]:
from flatsurvey.jobs import FlowDecompositions

We pull all the flow decompositions coming from triangles from the database that had undetermined components.

Note we need to `limit` the result to the most recent decompositions since there are way too many (>500k) in the database to download them all:

In [15]:
undetermined = cache.query(job=FlowDecompositions, surface_filter="vertices: { equalTo: 3 }", result_filter="undetermined: { notEqualTo: 0 }", limit=32)

Let's zoom in on one such decomposition:

In [16]:
undetermined = undetermined.nodes()[0]

Unforfunately, `flatsurf::FlowDecomposition` cannot be pickled currently so trying to recover the pickle fails:

In [17]:
undetermined['result']

{'error': 'only default constructible types can be handled by cereal but <class cppyy.gbl.flatsurf.FlowDecomposition<flatsurf::FlatTriangulation<eantic::renf_elem_class> > at 0x563721e96b20> is not default constructible; you might wrap your type into a smart pointer or make it default constructible',
 'description': 'FlowDecomposition with 0 cylinders, 0 minimal components and 1 undetermined components'}

But we can recover the surface and the direction that were used:

In [18]:
import pyflatsurf
surface = undetermined['surface']()
direction = undetermined['orientation']()

In [19]:
surface

Ngon([4, 8, 9])

And from this we can recover the decomposition:

In [20]:
pyflatsurf.flatsurf.makeFlowDecomposition(surface.flat_triangulation(), direction)

FlowDecomposition with 0 cylinders, 0 minimal components and 1 undetermined components